In [156]:
import pandas as pd 
import math

In [157]:
# Read in the data
df = pd.read_csv('../data/Corona_train.csv') 
print(df) 

          ID Sentiment                                        CoronaTweet
0      22979  Positive  I see all kinds of academics already whipping ...
1       9880  Negative  @HenrySmithUK can you raise with Boris please ...
2      35761  Negative  It s a confusing odd time for the shopping pub...
3      37968  Positive  Blog Summary: The Impact of COVID-19 on the Ca...
4      19709   Neutral  ??????? ??????? ???\r\r\nWaiting in a long Que...
...      ...       ...                                                ...
37859  20253   Neutral  DOUBLE TAP ??IF YOU REMEMBER BEING EMPLOYED?\r...
37860  38926  Positive  Struggling to understand why supermarkets or f...
37861  35889  Positive  For anyone interested in the checklist on cons...
37862  40641  Positive  Do you agree with the hypothesis that Indian s...
37863  26063   Neutral  In Richmond, local boutiques and thrift stores...

[37864 rows x 3 columns]


In [158]:
df["splitWords"] = df["CoronaTweet"][:].str.split(" ")
# print(df)
# empty set
set_of_words = set()
frequencies = dict() 

In [159]:
# Negative -> 0, Neutral -> 1, Positive -> 2 
#prob_j_0, prob_j_1, prob_j_2 
# prob_j_i = no of occurences of word j in text of class i + 1 / total no of words in class i + |V| 
# |V| = size of dictionary 



In [161]:
# computing frequencies 


SIZES = [0, 0, 0]

for index, row in df.iterrows():
    if (row["Sentiment"] == "Positive"):
        df["Sentiment"][index] = 2
    elif (row["Sentiment"] == "Neutral"):
        df["Sentiment"][index] = 1
    else:
        df["Sentiment"][index] = 0
    i = df["Sentiment"][index] # i is class 
    for word in row["splitWords"]:
            if (word == "")  : continue
            SIZES[i] += 1
            if (word in set_of_words):
            
                 frequencies[word][i] += 1
            else:
                set_of_words.add(word)
                frequencies[word] = [0,0,0]
        

SIZE = len(set_of_words) 
print(SIZE) 

C:\Users\Ashu Jain\AppData\Local\Temp\ipykernel_27728\2036564936.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentiment"][index] = 2
C:\Users\Ashu Jain\AppData\Local\Temp\ipykernel_27728\2036564936.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentiment"][index] = 0
C:\Users\Ashu Jain\AppData\Local\Temp\ipykernel_27728\2036564936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentiment"][index] = 1


135814


In [ ]:
# computing log probabilities 
probabilities = dict()
log_probabilities = dict()

for word in set_of_words:
    probabilities[word] = [0,0,0]
    log_probabilities[word] = [0,0,0] 
    for i in range(3):
        probabilities[word][i] = (frequencies[word][i] + 1) / (SIZES[i] + SIZE)
        log_probabilities[word][i] = math.log(probabilities[word][i])


In [ ]:

# for (word, prob) in log_probabilities.items():
#     print(word) 
#     if (len(word) == 0): print("found")
#     # break 
# # print(frequencies[""])

y_prob = [SIZES[0]/SIZE, SIZES[1]/SIZE, SIZES[2]/SIZE]
y_log_prob = [math.log(y_prob[0]), math.log(y_prob[1]), math.log(y_prob[2])]     


In [ ]:
def compute_canonical_log_prob(text, label, log_probabilities, set_of_words): # ignores P(x) in P(y | x) = P(x|y) * P(y)/ P(x) 
    word_list = text.split(" ")
    log_prob = 0 
    for word in word_list:
        if (word == "") : continue
        if (word in set_of_words):
            log_prob += log_probabilities[word][label] 
        else:
            log_prob -= math.log((SIZES[label] + SIZE))  # prob is 1 / (SIZES[label] + SIZE) 
    log_prob += y_log_prob[label] 
    return log_prob 

In [ ]:
def predictor(text,log_probabilities, set_of_words, SIZES):
    no_of_labels = 3
    for word,probs in log_probabilities.items():
        no_of_labels = len(log_probabilities[word]) 
        break 
    
    log_probs = [compute_canonical_log_prob(text, i, log_probabilities, set_of_words) for i in range(no_of_labels)] 
    max_prob = log_probs[0] 
    max_label = 0 

    for i in range(no_of_labels):
        if (log_probs[i] > max_prob):
            max_prob = log_probs[i]
            max_label = i
        elif (log_probs[i] == max_prob):
            max_label = max_label if (SIZES[max_label] > SIZES[i])  else i 
    return max_label


In [ ]:
def compute_accuracy(df, log_probabilities, set_of_words, SIZES): # input is a dataframe
    correct = 0 
    for idx,rows in df.iterrows():
        prediction = predictor(rows["CoronaTweet"], log_probabilities, set_of_words, SIZES)
        print(f"{prediction} {rows['Sentiment']}")
        if (prediction == rows["Sentiment"]):
            correct += 1
    return correct / len(df) 



In [162]:
training_accuracy = compute_accuracy(df, log_probabilities, set_of_words, SIZES)
print(training_accuracy)

2 2
0 0
2 0
1 2
1 1
0 1
2 2
2 2
2 2
1 2
0 0
1 1
2 2
0 0
0 0
0 1
0 0
1 1
1 1
0 0
2 0
2 1
0 0
0 0
0 0
0 1
0 1
2 2
0 2
2 1
1 2
0 0
0 0
0 0
0 2
0 0
0 0
0 0
2 0
0 2
2 1
2 0
0 0
2 2
2 2
2 2
0 2
2 2
2 0
0 0
2 1
2 0
2 0
2 2
2 2
2 0
2 2
0 2
2 1
0 0
0 0
0 0
2 2
2 2
0 0
2 2
2 2
2 2
0 0
0 0
0 2
0 0
1 1
2 2
0 2
0 0
0 0
2 0
1 2
2 2
2 2
2 2
0 0
0 0
0 2
2 2
0 0
2 0
0 0
2 2
2 2
0 2
0 0
0 0
1 1
2 2
0 0
2 1
2 2
0 2
1 1
0 0
2 2
2 2
2 1
2 1
0 0
2 0
2 2
0 0
0 2
0 2
2 0
2 1
0 1
0 0
0 2
0 0
2 0
1 0
2 1
2 2
0 0
0 2
0 2
2 2
2 2
1 1
2 2
2 2
2 0
1 1
1 1
0 0
0 0
2 2
2 2
0 0
0 0
0 0
2 2
0 0
2 2
2 2
1 2
2 2
0 0
1 2
2 2
2 2
2 2
0 2
2 1
0 0
2 0
2 2
2 2
0 2
2 2
0 0
1 1
0 0
2 2
2 2
2 2
2 2
0 0
0 0
0 2
0 1
0 0
2 2
0 2
2 0
2 0
2 2
0 0
1 1
0 0
0 2
0 1
0 0
2 2
0 1
0 0
2 0
2 2
0 2
2 2
1 1
0 0
2 2
2 2
2 1
2 2
2 0
0 0
0 0
0 1
0 0
0 2
2 2
0 0
0 2
2 2
0 2
2 0
2 2
2 2
0 0
0 0
2 2
2 0
2 0
0 0
2 2
2 2
2 2
2 0
2 2
2 2
2 2
0 0
0 0
0 0
2 2
2 2
2 2
0 1
2 2
0 0
2 2
2 2
0 0
2 0
0 2
0 0
0 0
2 2
1 2
0 2
0 0
0 0
2 2
2 2
2 1
2 2
0 0
0 0
2 2
